In [11]:
import os
import sys
root = '/home/mark/Documents/code/mdrp'
sys.path.append(root)

import numpy as np
from utils.nlp_mdrp_elastic import mdrp_prob_elastic
import cyipopt

from utils.convert import convert_instance
# from utils.nlp_mdrp import mdrp_prob
from utils.helpers import *

rng = np.random.default_rng(seed=0)

In [12]:
# HELPER FUNCTIONS FOR NOTEBOOK #
def get_mu(s_time,t_time,beta,N,k):
    '''
    computers average service rate as the expected value for the given
    problem isntance
    '''
    expected_lat = (s_time+t_time+(k/(1+beta*N*(0.1)))).mean(axis=0)
    print(expected_lat)
    return 1/expected_lat

# max_dph = 100
# min_dph = 5
# alpha = lambda x : (1/max_dph) + ((1/min_dph)-(1/max_dph))*x 
# alpha_inv = lambda x: (x-(1/max_dph))/((1/min_dph)-(1/max_dph))

In [13]:
# Parameters for toy problem (INELASTIC)
# Units and rates are in hours
n_orders = 20
n_modes = 3
speeds = [1,2,0.3]
max_dph = 100
min_dph = 10
gamma = 2

prob_setup = {}

# same service time
prob_setup['s_time'] = np.ones((n_orders,n_modes))*(5/60) 
# travel time scaled by speeds
base_t_time = rng.random(n_orders)*30
# base_t_time = 15*np.ones()
prob_setup['t_time'] = np.array([base_t_time/speeds[0],
                                 base_t_time/speeds[1],
                                 base_t_time/speeds[2]]).T/60
# will be changed manually after
prob_setup['cost'] = np.ones((n_orders,n_modes))*(1) 
# same distribution for now
prob_setup['beta'] = np.ones((n_orders,n_modes))
# prob_setup['beta'] = rng.random((n_orders,n_modes))
# Set N so that setting makes sense
prob_setup['N'] = np.array([2,10,20])
# time radius considered for beta
prob_setup['k'] = np.ones(n_modes)*10/60
# compute mu which depends on N as well
prob_setup['mu'] = get_mu(prob_setup['s_time'],
                          prob_setup['t_time'],
                          prob_setup['beta'],
                          prob_setup['N'],
                          prob_setup['k'])

#ELASTIC DEMAND 
# properties of linear alpha function
# this controls the intrinsic value of ordering, higher means more demand
# NOTE: should be tuned to somewhere around latency we expect to wait (do later)
prob_setup['gamma'] = gamma
prob_setup['min_tau'] = 1e-9
# min_dph = prob_setup['min_tau']/prob_setup['gamma']

#  delta needs demand as well when defined this way
prob_setup['delta'] = (1/min_dph)-(1/max_dph)
prob_setup['alpha_0'] = (1/max_dph)

# constraints
prob_setup['max_rho'] = 0.9
prob_setup['max_cost'] = 10000 # irrelevant 
prob_setup['demand'] = 0.99
prob_setup['scale'] = True
# differents setups tested
# N_values = ([100,10,20],[80,10,35])
# # costs are set arbitrarily large to not allow for cars
# costs = ([10,200000,200000],[1,1,1])

[0.47839816 0.29475463 0.99280867]


In [14]:
# prob_setup['N'] = N_values[1]
# prob_setup['cost'][:,0] = costs[1][0]
# prob_setup['cost'][:,1] = costs[1][1]
# prob_setup['cost'][:,2] = costs[1][2]

mdrp = mdrp_prob_elastic(prob_setup)
lb, ub, cl, cu = mdrp.get_bounds()
x0 = mdrp.guess_init()

nlp = cyipopt.Problem(
    n=len(x0),
    m=len(cl),
    problem_obj=mdrp,
    lb=lb,
    ub=ub,
    cl=cl,
    cu=cu
    )

# nlp.addOption('derivative_test', 'second-order')
nlp.add_option('mu_strategy', 'adaptive')
nlp.add_option('tol', 1e-4)

nlp.set_problem_scaling(
    obj_scaling = -1
)

nlp.add_option('nlp_scaling_method', 'user-scaling')
# we are maximizing
x, info = nlp.solve(x0)


This is Ipopt version 3.14.11, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     1600
Number of nonzeros in inequality constraint Jacobian.:      240
Number of nonzeros in Lagrangian Hessian.............:     3240

Total number of variables............................:       80
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       80
                     variables with only upper bounds:        0
Total number of equality constraints.................:       20
Total number of inequality constraints...............:        3
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        3
        inequality constraints with only upper bounds:        0

Objective value at iteration #0 is - 815.827
(60, 60) (60, 60)
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  8.158270

In [15]:
# x  = mdrp.guess_init()

print(mdrp.objective(x).shape)
print(mdrp.constraints(x).shape)
# print(mdrp.gradient(x).shape)
# print(mdrp.jacobian(x).shape)
# print(mdrp.hessian(x, np.ones(mdrp.n+mdrp.n),1).shape)

print(mdrp.objective(x))
print(mdrp.constraints(x))
# print(mdrp.gradient(x))
# print(mdrp.jacobian(x))
# print(mdrp.hessian(x, np.ones(mdrp.n+mdrp.n),1))

()
(23,)
805.9219034071933
[ 8.96891998e-01  7.13638599e-01  8.99143798e-01  3.15924531e-08
  4.70543573e-08  2.31408595e-08  7.26617495e-08  4.67724708e-08
  2.36812168e-08  3.08042347e-08  5.42692757e-08  4.45195845e-08
  1.76034272e-08  4.08708796e-08  2.83012502e-08  2.55555708e-08
 -1.97785877e-09  4.77331024e-08  1.48617185e-08  3.76846745e-08
  2.21923746e-08  3.48106819e-08  1.43426098e-08]


In [16]:
flow, tau = mdrp.split_x(x)
print('util: ',mdrp.get_util(flow))
print('price:',tau)
print('obj: ',mdrp.objective(x))
print('Total Utilization: %.2f'%(100*flow.sum()/(mdrp.demand*mdrp.n)))
# print('Avg mode latency: ',mdrp.lat.mean(axis=0)*60)

util:  [0.896892  0.7136386 0.8991438]
price: [ 8.80976827 14.58476118 18.01724664 18.02951163  5.98358795  4.57138925
  9.24145209  7.40245484 10.17024252  4.19463071  5.86959608 18.16498418
  5.49295018 17.90958382  7.19565326 16.25588328  5.24905322 10.38920894
 14.05037275 12.33555886]
obj:  805.9219034071933
Total Utilization: 88.77


In [17]:
print(flow.sum(axis=1))
print(mdrp.demand)

[2.3295091  2.50281038 2.52516102 2.54279527 2.14993698 1.85730931
 2.36492708 2.22645033 2.4203663  1.79434634 2.17613912 2.53262525
 1.98159849 2.54802171 2.29766394 2.49415987 2.0342014  2.37328564
 2.5066125  2.41605311]
2.595127046821611


Intermediate steps

- objective (easy)

- gradient
    * get $\frac{d\tau_{i',j'}}{dx_{i,j}}$
    * get $d_{i,j}$

In [18]:
# eval_setup = {}
# eval_setup['max_dph'] = 100
# eval_setup['min_dph'] = 5
# eval_setup['min_price'] = 10
# eval_setup['mode_names'] = ['cars','drones','droids']
# eval_setup['speeds'] = [1,2,0.3]

# print('Evaluation')
# print('Mean delivery rate mu: ',(1/mdrp.mu)*60)
# eval_sol(x,mdrp,eval_setup)

In [19]:
# prob_setup['N'] = N_values[1]
# prob_setup['cost'][:,0] = costs[1][0]
# prob_setup['cost'][:,1] = costs[1][1]
# prob_setup['cost'][:,2] = costs[1][2]
# mdrp = mdrp_prob(prob_setup)
# lb, ub, cl, cu = mdrp.get_bounds()
# x0 = mdrp.guess_init()

# nlp = cyipopt.Problem(
#     n=len(x0),
#     m=len(cl),
#     problem_obj=mdrp,
#     lb=lb,
#     ub=ub,
#     cl=cl,
#     cu=cu
#     )

# # nlp.addOption('derivative_test', 'second-order')
# nlp.add_option('mu_strategy', 'adaptive')
# nlp.add_option('tol', 1e-4)

# x, info = nlp.solve(x0)

In [20]:
# eval_setup = {}
# eval_setup['max_dph'] = 100
# eval_setup['min_dph'] = 5
# eval_setup['min_price'] = 10
# eval_setup['mode_names'] = ['cars','drones','droids']
# eval_setup['speeds'] = [1,2,0.3]

# print('Evaluation')
# print('Mean delivery rate mu: ',(1/mdrp.mu)*60)
# eval_sol(x,mdrp,eval_setup)